In [2]:
!pip install sentence_transformers
!pip install -q pyvi
!pip install gdown


from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from pyvi.ViTokenizer import tokenize
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm 
import ast
import re
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 69.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.9 MB/s eta 0:00:00


In [3]:
#!gdown "https://drive.google.com/uc?id=17OBpx5TFZhk7JEuyvUvG808LCtwFZ_dL"
# https://drive.google.com/file/d/17OBpx5TFZhk7JEuyvUvG808LCtwFZ_dL/view?usp=sharing
# https://drive.google.com/file/d/11m4ugPOa3N6AQ-Y5rZXQewukf3oyYFDS/view?usp=sharing
# https://drive.google.com/file/d/1KyvrlAYCFzv_kvh0orz4x3x-i6o1CmmA/view?usp=drive_link

# !gdown --id 1HU7ayOiGOEZL4LocxkP4nkGS-QbdnDw8 --output checkpoint-10000.zip #pho_3

# !unzip /kaggle/working/checkpoint-10000.zip

!gdown --id 1_Pq9VvehWr3d5_2wSkjKf_EVZShyUD8q --output checkpoint-best4.zip
!unzip /kaggle/working/checkpoint-best4.zip

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1_Pq9VvehWr3d5_2wSkjKf_EVZShyUD8q
From (redirected): https://drive.google.com/uc?id=1_Pq9VvehWr3d5_2wSkjKf_EVZShyUD8q&confirm=t&uuid=a7c2f2d8-6475-4e9a-bc23-ab61dbc3b107
To: /kaggle/working/checkpoint-best4.zip
100%|██████████████████████████████████████| 1.26G/1.26G [00:20<00:00, 62.2MB/s]
Archive:  /kaggle/working/checkpoint-best4.zip
  inflating: checkpoint-24000/config_sentence_transformers.json  
  inflating: checkpoint-24000/config.json  
  inflating: checkpoint-24000/model.safetensors  
  inflating: checkpoint-24000/tokenizer_config.json  
  inflating: checkpoint-24000/special_tokens_map.json  
  inflating: checkpoint-24000/added_tokens.json  
  inflating: checkpoint-24000/vocab.txt  
  inflating: ch

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer('/kaggle/working/checkpoint-24000', trust_remote_code=True)
model = model.to(device)


In [5]:
import numpy as np

def load_embeddings_and_cids(embeddings_path, cids_path):
    embeddings = np.load(embeddings_path)
    print("Embeddings shape:", embeddings.shape) 
    cids = np.load(cids_path)
    print("\nCIDs shape:", cids.shape)

    return embeddings, cids

embeddings, cids = load_embeddings_and_cids("/kaggle/input/new-bi-enocde/embeddings_embeddings.npy", "/kaggle/input/new-bi-enocde/embeddings_cids.npy")

Embeddings shape: (261597, 768)

CIDs shape: (261597, 1)


In [7]:
import pandas as pd

corpus = pd.read_csv('/kaggle/input/fix-tokenize-corpus/corpus.csv')
last_samples = pd.read_csv('/kaggle/input/split-data/test.csv')
print(last_samples.shape)  # Đảm bảo đúng 9,456 mẫu
last_samples.head()

(10000, 4)


,question,context,cid,qid
0,Phó Tổng Giám đốc Ngân hàng Chính sách xã hội ...,['Áp dụng chế độ tiền lương và phụ cấp quy địn...,[140864],70867
1,Ai có thẩm quyền quyết định thành lập Hội đồng...,['Thành lập Hội đồng\n1. Bộ trưởng Bộ Y tế ra ...,[62339],813
2,Thời hiệu xử phạt đối với nhà xuất bản thực hi...,['Mức phạt tiền và thẩm quyền xử phạt\n...\n4....,[63171],40392
3,"Việc ký kết, thực hiện thỏa thuận quốc tế nhân...",['Báo cáo tình hình ký kết và thực hiện thỏa t...,[157761],85946
4,Đề án sử dụng tài sản công tại đơn vị sự nghiệ...,"['""Điều 44. Đề án sử dụng tài sản công tại đơn...",[95397],55607


In [10]:
from tqdm import tqdm

# Cấu hình batch size
BATCH_SIZE = 1
result = []
print("start")
# Mở file một lần
with open("query_rand.txt", "w") as f:
    # Duyệt qua last_samples theo batch
    for i in tqdm(range(0, len(last_samples), BATCH_SIZE)):
        # Lấy batch hiện tại
        batch = last_samples.iloc[i:i + BATCH_SIZE]
        
        # Chuẩn bị dữ liệu cho batch
        qids = batch['qid'].tolist()
        questions = batch['question'].tolist()
        tokenizer_sent = [tokenize(sent) for sent in questions]
        
        # Encode batch
        query_embeddings = model.encode(tokenizer_sent, show_progress_bar=False)
        
        # Thực hiện semantic search cho batch
        batch_hits = [util.semantic_search(query_embedding, embeddings, top_k=50)[0] for query_embedding in query_embeddings]
        
        # Lưu kết quả
        for qid, hits in zip(qids, batch_hits):
            f.write(f"{qid} ")
            for hit in hits:
                corpus_id = hit['corpus_id']
                cid = cids[corpus_id]
                result.append(cid)
                f.write(f"{cid[0]} ") 
            f.write("\n")


start


100%|██████████| 10000/10000 [1:21:35<00:00,  2.04it/s]


In [6]:
# from scipy.spatial.distance import cosine

In [14]:
# query = train.loc[train['qid'] == 87184 ]['question'].values[0]
# context_pos = corpus.loc[corpus['cid'] == 10682 ]['text'].values[0]
# embeddings1 = model.encode(tokenize(query), show_progress_bar = False)
# embeddings2 = model.encode(tokenize(context_pos).strip('""'), show_progress_bar = False)
# similarity = 1 - cosine(embeddings1, embeddings2)  # Vì `scipy.cosine` trả về cosine distance (1 - similarity)
# print(f"Cosine Similarity: {similarity}") #7x

Cosine Similarity: 0.438807262467944


In [77]:
# query = tokenize(train.loc[train['qid'] == 136414    ]['question'].values[0])
# #query = query.strip("?")
# query_embedding = model.encode(query, show_progress_bar=False)
# hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]

# for hit in hits:  
#     corpus_id = hit['corpus_id']
#     cid = cids[corpus_id]
#     score = hit['score']
#     print(f"CID: {cid}, Score: {score:.4f}")
#     #result.append((cid, score))


CID: [545516], Score: 0.6803
CID: [20871], Score: 0.6715
CID: [111880], Score: 0.6605
CID: [184391], Score: 0.6458
CID: [494797], Score: 0.6286
CID: [28968], Score: 0.6098
CID: [72293], Score: 0.6044
CID: [49463], Score: 0.6040
CID: [20877], Score: 0.5969
CID: [247035], Score: 0.5954
CID: [89968], Score: 0.5944
CID: [26031], Score: 0.5910
CID: [26030], Score: 0.5874
CID: [863], Score: 0.5847
CID: [499324], Score: 0.5759
CID: [476202], Score: 0.5740
CID: [449164], Score: 0.5716
CID: [860], Score: 0.5714
CID: [49445], Score: 0.5647
CID: [194600], Score: 0.5593
CID: [575083], Score: 0.5528
CID: [499325], Score: 0.5501
CID: [44605], Score: 0.5468
CID: [505739], Score: 0.5387
CID: [203713], Score: 0.5311
CID: [38080], Score: 0.5292
CID: [47791], Score: 0.5185
CID: [575079], Score: 0.5140
CID: [20858], Score: 0.5116
CID: [588966], Score: 0.5103
CID: [42684], Score: 0.5044
CID: [49470], Score: 0.5023
CID: [109174], Score: 0.5006
CID: [12074], Score: 0.4975
CID: [47786], Score: 0.4972
CID: [58